# 3D reconstruction using 2 image

Key References
- https://www.opencvhelp.org/tutorials/advanced/reconstruction-opencv/

In [ ]:
from __future__ import annotations

import os
import random as rd
from typing import TYPE_CHECKING

import cv2 as cv
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pyrender
import scipy.spatial.transform as sci_trans

from project_3d_reconstruction.rendering import render_helper as rh

if TYPE_CHECKING:
    from mpl_toolkits.mplot3d.axes3d import Axes3D

import pyvista as pv
import trimesh

from project_3d_reconstruction.point_cloud_to_mesh import (
    point_cloud_to_mesh as pcloud,
)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Generate simple scene

In [ ]:
FILE_PATH = os.path.abspath(
    os.path.join(
        os.path.abspath("pyrender_basic_reconstruction.ipynb"), "..", "..", "mesh_files"
    )
)
OBJECTS = [
    # os.path.join(FILE_PATH, "pyrender_examples", "drill.obj"),
    os.path.join(FILE_PATH, "coral_4", "untitled.dae"),
    # os.path.join(FILE_PATH, "duck", "duck5.obj"),
    # os.path.join(FILE_PATH, "cat", "watermelon.obj"),
]

renderer = rh.RenderHelper()

# Load in mesh files
names = []
for file_path in OBJECTS:
    print(f"Loading {file_path}")
    name = file_path.split("\\")[-2]  # warning slash is different for linux vs windows
    names.append(name)
    renderer.loadFromPath(file_path, name)
print("Done loading")

# Place them in the world
renderer.addFromMeshDict("coral_4", rh.positionOnly(0, 0, 0))

Loading c:\Code\Project3DReconstruction\mesh_files\coral_4\untitled.dae
Done loading


# test image from two perspectives.

In [ ]:
# it got annoying making edits in render helper so its here now (TODO move it back)
# NOTE z is out, y is up (image y might be flipped), x is right
def toOriginPose2(x, y, z):
    # azimuth from xz plane
    az_rad = np.arctan2(x, z)
    # rotation about y axis
    Ry = np.array(
        [
            [np.cos(az_rad), 0, np.sin(az_rad)],
            [0, 1, 0],
            [-np.sin(az_rad), 0, np.cos(az_rad)],
        ]
    )

    el_rad = np.arctan2(-y, np.sqrt(x**2 + z**2))
    Rx = np.array(
        [
            [1, 0, 0],
            [0, np.cos(el_rad), -np.sin(el_rad)],
            [0, np.sin(el_rad), np.cos(el_rad)],
        ]
    )
    R = Ry @ Rx
    Rstack = np.hstack((R, np.array([[x, y, z]]).T))
    return np.vstack((Rstack, np.array([[0, 0, 0, 1]])))


# wrapper for easy even spacing
def toOriginPose3(az_rad, el_rad, radius):
    y = radius * np.sin(el_rad)
    rproj = radius * np.cos(el_rad)
    x = rproj * np.sin(az_rad)
    z = rproj * np.cos(az_rad)
    return toOriginPose2(x, y, z)

# save many images


In [ ]:
# Render many views in a spiral pattern
poses = []
images = []
numPoses = 40
azSpacing = 2.0 * np.pi / numPoses
elSpacing = np.pi * 2.0 / (3 * numPoses)
for i in range(numPoses):
    pose1 = toOriginPose3(
        azSpacing * 2 * i, -np.pi / 3.0 + elSpacing * i, 1
    )  # 0.5) for drill #2.5)#for duck
    renderer.moveCamera(pose1)
    renderer.render(show_image=False, image_filename="test" + str(i) + ".png")
    poses.append(pose1)
    # TODO optimize this out if we're low on memory
    images.append(cv.imread("test" + str(i) + ".png"))

# actual extrinsics
# print(pose1)
# print(pose2)
# relativePose = np.matmul(np.linalg.inv(pose1), pose2)
# print(relativePose)

gray_0 = cv.cvtColor(images[0], cv.COLOR_BGR2GRAY)
gray_1 = cv.cvtColor(images[1], cv.COLOR_BGR2GRAY)

# ifig, iaxs = plt.subplots(ncols=numPoses, figsize=(8, 8))
# for i in range(numPoses):
#     iaxs[i].imshow(images[i], origin="lower")
# ifig.tight_layout()

# Detect SIFT features

In [ ]:
def opponentSIFT(img):
    # Step 1 convert to opponent color space
    # TODO optimize
    B = img[:, :, 0]
    G = img[:, :, 1]
    R = img[:, :, 2]

    O1 = np.divide((R - G), np.sqrt(2))
    O2 = np.divide((R + G - 2 * B), np.sqrt(6))
    O3 = np.divide((R + G + B), np.sqrt(3))
    # visually check opponent color space
    # cv.imwrite('sift_keypointsO1.jpg',np.uint8(O1))
    # cv.imwrite('sift_keypointsO2.jpg',np.uint8(O2))
    # cv.imwrite('sift_keypointsO3.jpg',np.uint8(O3))

    # Step 2 use Harris-Laplace point detector on intensity channel (o3)
    # TODO use a real point detector or figure out what parameters to use with cv SIFT
    # use this space to specify additional parameters
    sift = cv.SIFT_create()
    # sift = cv.SIFT_create(nfeatures=1000)

    kp = sift.detect(np.uint8(O3), None)

    # Step 3 compute descriptors for each opponent channel
    _, des1 = sift.compute(np.uint8(O1), kp)
    _, des2 = sift.compute(np.uint8(O2), kp)
    _, des3 = sift.compute(np.uint8(O3), kp)

    # combine into one large descriptor
    des = np.concatenate((des1, des2, des3), axis=1)

    return kp, des


kpList = []
desList = []
for i in range(numPoses):
    kp, des = opponentSIFT(images[i])
    kpList.append(kp)
    desList.append(des)

image_kp_1 = cv.drawKeypoints(
    gray_0, kpList[0], images[0], flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
)
image_kp_2 = cv.drawKeypoints(
    gray_1, kpList[1], images[1], flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
)

ifig, iaxs = plt.subplots(ncols=2, figsize=(8, 8))
iaxs[0].imshow(image_kp_1)
iaxs[1].imshow(image_kp_2)

# Match features (brute-force)

In [ ]:
bf = cv.BFMatcher()

# matches between index i and i+1
bfMatchesList = []
for i in range(numPoses - 1):
    bf_matches = bf.knnMatch(desList[i], desList[i + 1], k=2)
    good = []

    for m, n in bf_matches:
        if m.distance < 0.8 * n.distance:
            good.append([m])
    bfMatchesList.append(good)

In [ ]:
show_index = 18
image_matches = cv.drawMatchesKnn(
    images[show_index],
    kpList[show_index],
    images[show_index + 1],
    kpList[show_index + 1],
    bfMatchesList[show_index],
    None,
    flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS,
)


ifig, iax = plt.subplots()
iax.imshow(image_matches)
ifig.tight_layout()
print(len(bfMatchesList[show_index]))

1294


# Estimate essential matrix, filter points with ransac

In [ ]:
# see pyrender script for intrinsic camera params
intrinsic_mat = np.array([[2048, 0, 1024], [0, 2048, 1024], [0, 0, 1]])

ransacMatches = []
fMatrices = []
for i in range(numPoses - 1):
    good = bfMatchesList[i]
    kp1 = kpList[i]
    kp2 = kpList[i + 1]
    points_1 = np.int32([kp1[m.queryIdx].pt for [m] in good]).reshape(-1, 1, 2)
    points_2 = np.int32([kp2[m.trainIdx].pt for [m] in good]).reshape(-1, 1, 2)

    fMatrix, mask = cv.findFundamentalMat(
        points_1,
        points_2,
        method=cv.FM_RANSAC,
        ransacReprojThreshold=3,
        confidence=1 - 1e-6,
    )

    postRansac = []
    if mask is not None:
        for i in range(len(mask)):
            if mask[i]:
                postRansac.append(good[i])
    else:
        print("warning no points recovered for " + str(i))
    ransacMatches.append(postRansac)
    fMatrices.append(fMatrix)

In [ ]:
essential_mat = intrinsic_mat.T @ fMatrices[0] @ intrinsic_mat

good = ransacMatches[0]
kp1 = kpList[0]
kp2 = kpList[1]
points_1 = np.int32([kp1[m.queryIdx].pt for [m] in good]).reshape(-1, 1, 2)
points_2 = np.int32([kp2[m.trainIdx].pt for [m] in good]).reshape(-1, 1, 2)

_, est_rot, est_trans, _ = cv.recoverPose(
    essential_mat, points_1, points_2, intrinsic_mat
)

est_extrinsic = np.hstack((est_rot, est_trans))

# act_rot = sci_trans.Rotation.align_vectors(dir_2, dir_1)[0].as_matrix()
# act_extrinsic = np.hstack((act_rot, np.reshape(dir_2 - dir_1, (-1, 1))))
# actual extrinsics
# print(pose1)
# print(pose2)
relativePose = np.matmul(np.linalg.inv(poses[0]), poses[1])
# print(relativePose)
act_extrinsic = relativePose[0:3, :]
print(est_extrinsic)
print(act_extrinsic)

origin = np.hstack((np.eye(3), np.zeros((3, 1))))

all_extrinsics = [origin, act_extrinsic, est_extrinsic]
base_colors = [
    mpl.colors.to_rgb("tab:blue"),
    mpl.colors.to_rgb("tab:green"),
    mpl.colors.to_rgb("tab:red"),
]
cfig, cax = plt.subplots(subplot_kw={"projection": "3d"})
for mat, color in zip(all_extrinsics, base_colors, strict=False):
    pos = mat[:, 3]
    for column, weight in zip(mat[:, :3].T, [1, 0.8, 0.5], strict=False):
        cax.quiver(*pos, *column, color=np.array(color) * weight)

cfig.tight_layout()
cax.set_xlim3d(-1, 1)
cax.set_ylim3d(-1, 1)
cax.set_zlim3d(-1, 1)
# cax.voxels(voxelarray, facecolors=colors, edgecolors=colors);

# note: rotation vectors stay aligned and translation is close enough (within scale factor)
# NOTE the chirality is messed up. pyrender camera has an inverted y axis. estimated translation will have the opposite direction

[[ 0.9554036   0.27377234  0.11069179 -0.86398621]
 [-0.27031095  0.96169977 -0.04544818  0.44675411]
 [-0.11889472  0.01350015  0.99281508 -0.23224684]]
[[ 0.95105652  0.25916346  0.16830272  0.16830272]
 [-0.26761657  0.96308139  0.02925072  0.02925072]
 [-0.1545085  -0.07285969  0.98530127 -0.01469873]]


(-1.0, 1.0)

In [ ]:
# turn feature points into 3D points using actual extrinsic matrix
points3DList = []
quality3DPointsList = []
highErrorPts = 0
for i in range(numPoses - 1):
    relativePose = np.matmul(np.linalg.inv(poses[i]), poses[i + 1])
    R = relativePose[0:3, 0:3]
    T = relativePose[0:3, 3]

    kp1 = kpList[i]
    kp2 = kpList[i + 1]
    good = ransacMatches[i]

    points_1_pr = np.int32([kp1[m.queryIdx].pt for [m] in good]).reshape(-1, 1, 2)
    points_2_pr = np.int32([kp2[m.trainIdx].pt for [m] in good]).reshape(-1, 1, 2)
    f = 2048.0
    points3D = np.zeros((len(points_1_pr), 3))
    currentPose = poses[i]
    for index in range(len(points_1_pr)):
        point1 = points_1_pr[index]
        point2 = points_2_pr[index]
        # NOTE inverting y axis here
        # also converting from pixels to camera frame
        ray1 = np.array([point1[0][0] - 1024, 1024 - point1[0][1], f])  # *a
        ray2 = R @ np.array([point2[0][0] - 1024, 1024 - point2[0][1], f])
        ray1 = ray1 / np.linalg.norm(ray1)
        ray2 = ray2 / np.linalg.norm(ray2)
        skewSegment = np.cross(ray1, ray2)
        skewSegment = skewSegment / np.linalg.norm(skewSegment)
        A = np.column_stack([ray1, -ray2, skewSegment])
        coeffs = np.linalg.solve(A, T)

        p13d = coeffs[0] * ray1
        p23d = T + coeffs[1] * ray2
        midpoint = (p13d + p23d) / 2.0

        # convert to world coordinates
        fourVec = np.concatenate([midpoint, [1]])
        point_worldCoords = (currentPose @ fourVec)[0:3]
        points3D[index] = point_worldCoords

        # warning, arbitrary threshold for filtering high error points
        if coeffs[2] / (np.linalg.norm(coeffs)) < 1e-2:
            quality3DPointsList.append(point_worldCoords)

    points3DList.append(points3D)
print(points3DList[0])
print(points3DList[1])

[[ 0.1526175   0.04640891 -0.25359814]
 [ 0.14773066  0.05712462 -0.18378115]
 [ 0.15215749  0.0656781  -0.25708157]
 ...
 [-0.18209923 -0.04093154 -0.23561221]
 [-0.17659141 -0.07751344 -0.23867718]
 [-0.18203924 -0.06023267 -0.24103459]]
[[ 0.15362999 -0.00089146 -0.20089768]
 [ 0.14975312  0.00631723 -0.2044812 ]
 [ 0.09249056  0.03568722 -0.46834355]
 ...
 [-0.20811178 -0.02238656 -0.30682372]
 [-0.21068337 -0.01582028 -0.30874559]
 [-0.21088518 -0.02911673 -0.29467801]]


In [ ]:
points3D = points3DList[18]
print(points3D[0])
# problem: sift feature points aren't always corners, not all corners make it to the final list either

cfig, cax = plt.subplots(subplot_kw={"projection": "3d"})
cax.scatter(points3D[:, 0], points3D[:, 1], points3D[:, 2])
cfig.tight_layout()

[ 0.08856319 -0.01959212  0.00792353]


In [ ]:
# all pts
cfig, cax = plt.subplots(subplot_kw={"projection": "3d"})
for points3D in points3DList:
    cax.scatter(points3D[:, 0], points3D[:, 1], points3D[:, 2])
cfig.tight_layout()

In [ ]:
# # actual mesh
# coralFile = os.path.join(FILE_PATH, "coral_4", "untitled.dae")
# mesh = trimesh.load(coralFile, force="mesh")
# pc1 = pcloud.generatePointCloud(mesh, noise=0.0)
# clusters1 = pcloud.extractClusters(pc1, distance_threshold=0.5)

# # Plot it
# pl1 = pv.Plotter(shape=(1, 2))
# pl1.add_title("Point Cloud of 3D Surface")
# pl1.add_mesh(pc1)
# pl1.subplot(0, 1)
# pl1.add_title("Reconstructed Surface")
# for cluster in clusters1:
#     surf = pcloud.pointsToSurface(cluster)
#     pl1.add_mesh(surf, color=True, show_edges=True)

# uncomment this to show
# pl1.show()

In [ ]:
flattenedPointsList = np.array(quality3DPointsList)
# flattenedPointsList = np.concatenate(points3DList)
print(flattenedPointsList)
print(numPoses)
reconstructedMesh = pcloud.pointsToMesh(flattenedPointsList, "open3d_alpha", 0.1)

# Plot it
pl = pv.Plotter(shape=(1, 2))
pl.add_title("Point Cloud of 3D Surface")
pl.add_mesh(flattenedPointsList)
pl.subplot(0, 1)
pl.add_title("Reconstructed Surface")

pl.add_mesh(reconstructedMesh, color=True, show_edges=True)

# uncomment this to show
pl.show()

[[ 0.1526175   0.04640891 -0.25359814]
 [ 0.14773066  0.05712462 -0.18378115]
 [ 0.15215749  0.0656781  -0.25708157]
 ...
 [-0.19217237  0.03611609 -0.20578269]
 [-0.190068    0.03676289 -0.21074208]
 [-0.19543211  0.04003542 -0.21145896]]
40


Widget(value='<iframe src="http://localhost:55064/index.html?ui=P_0x1f85f1a6290_17&reconnect=auto" class="pyvi…

In [ ]:
# calculate I over U

# actual mesh
coralFile = os.path.join(FILE_PATH, "coral_4", "untitled.dae")
actualmesh = trimesh.load(coralFile, force="mesh")

# fudging the scale. somewhere along the line this gets messed up, and the signs all got flipped?
scalar = 2.5
vertices = [
    np.array([-scalar * v[0], -scalar * v[1], -scalar * v[2]])
    for v in actualmesh.vertices
]

actualmesh2 = pcloud.pointsToMesh(vertices, "open3d_alpha", 0.02 * scalar)

# Plot it
pl = pv.Plotter(shape=(1, 1))
pl.add_title("Point Cloud of 3D Surface")
pl.add_mesh(reconstructedMesh, color=[255, 0, 0], show_edges=True, opacity=0.5)
pl.add_mesh(actualmesh2, color=[0, 0, 255], show_edges=True, opacity=0.5)

# uncomment this to show
pl.show()

Widget(value='<iframe src="http://localhost:55064/index.html?ui=P_0x1f86f3ff790_28&reconnect=auto" class="pyvi…

In [ ]:
# meshes are not watertight
# temporary fix could be to increase alpha until they are

trimesh.repair.fix_winding(actualmesh)
trimesh.repair.fix_inversion(actualmesh)
trimesh.repair.fill_holes(actualmesh)
trimesh.repair.fix_normals(actualmesh)

# check with self to see if its watertight
# ratio = pcloud.intersectionOverUnion(actualmesh,actualmesh)
ratio = pcloud.intersectionOverUnion(actualmesh2, reconstructedMesh)
print(ratio)

ValueError: Not all meshes are volumes!